# Smart Lock Market Analysis

## Project Objectives
- Analyze the smart lock market segment
- Identify key players and market characteristics
- Evaluate brand performance through multiple lenses

## Data Source
- Platform: Flipkart
- Database: smart_lock.db
- Analysis Focus: Brand Distribution, Pricing, Ratings

In [1]:
# Import required libraries
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plot styles
sns.set_theme(style="darkgrid")  # Seaborn styling
sns.set_palette('viridis')  # Set Seaborn color palette

In [2]:
# Database Connection
def connect_database(db_path='smart_lock.db'):
    """Establish a connection to the SQLite database."""
    try:
        conn = sqlite3.connect(db_path)
        print("Database connection successful.")
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

conn = connect_database()

Database connection successful.


In [3]:
def get_market_overview_metrics(conn):
    """Calculate key market overview metrics."""
    # Define the queries
    queries = {
        'total_brands': "SELECT COUNT(DISTINCT brand) AS total_brands FROM products",
        'total_products': "SELECT COUNT(*) AS total_products FROM products",
        'avg_price': "SELECT ROUND(AVG(price), 2) AS avg_price FROM products",
        'avg_rating': "SELECT ROUND(AVG(rating), 2) AS avg_rating FROM products WHERE rating IS NOT NULL"
    }
    
    # Execute the queries
    metrics = {}
    for key, query in queries.items():
        metrics[key] = pd.read_sql_query(query, conn).iloc[0][0]
    
    # Calculate average SKUs per brand
    metrics['avg_skus_per_brand'] = round(metrics['total_products'] / metrics['total_brands'], 2)
    
    return metrics

# Fetch and print market metrics
market_metrics = get_market_overview_metrics(conn)
print(market_metrics)

{'total_brands': 108, 'total_products': 247, 'avg_price': 9866.56, 'avg_rating': 3.5, 'avg_skus_per_brand': 2.29}


C:\Users\Admin\AppData\Local\Temp\ipykernel_15500\1190177226.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  metrics[key] = pd.read_sql_query(query, conn).iloc[0][0]


## Market Overview

| Metric | Value |
|--------|-------|
| Number of Brands | 108 |
| Total Products | 247 |
| Average SKUs per Brand | 2.29 |
| Average Price | ₹9,866.56 |
| Average Rating | 3.5 ⭐ |

In [4]:
def analyze_brand_distribution(conn):
    """Analyze product distribution across top brands using a bar chart."""
    # Query to calculate brand distribution
    query = """
    SELECT 
        brand, 
        COUNT(*) AS product_count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products), 2) AS market_share
    FROM products
    GROUP BY brand
    ORDER BY product_count DESC
    LIMIT 10
    """
    brand_dist = pd.read_sql_query(query, conn)
    
    # Query to calculate the total number of products
    total_products_query = "SELECT COUNT(*) AS total_products FROM products"
    total_products = pd.read_sql_query(total_products_query, conn).iloc[0]['total_products']
    
    # Create bar chart
    fig = px.bar(
        brand_dist, 
        x='brand', 
        y='product_count', 
        text='market_share', 
        title=f'Top 10 Brands: Product Distribution (Total Products: {total_products})',
        labels={'product_count': 'Number of Products', 'brand': 'Brand'},
        hover_data={'market_share': True}
    )
    
    # Enhance the visualization
    fig.update_traces(
        texttemplate='%{text}%', 
        textposition='inside',
        textfont_size=12,  # Adjust font size if needed
        textangle=0  # Optional: to keep the text horizontal
    )
    fig.update_layout(
        xaxis_title="Brand",
        yaxis_title="Number of SKUs",
        uniformtext_minsize=12,
        uniformtext_mode='hide'
    )
    fig.show()

    return brand_dist, total_products

# Call the function
brand_dist, total_products = analyze_brand_distribution(conn)

In [5]:
def analyze_price_distribution(conn):
    """Analyze SKU distribution by price bands."""
    query = """
    SELECT
        CASE 
            WHEN price < 3000 THEN 'Below INR 3000'
            WHEN price BETWEEN 3000 AND 4999 THEN 'INR 3000-4999'
            WHEN price BETWEEN 5000 AND 9999 THEN 'INR 5000-9999'
            WHEN price BETWEEN 10000 AND 14999 THEN 'INR 10000-14999'
            WHEN price BETWEEN 15000 AND 19999 THEN 'INR 15000-19999'
            ELSE 'Greater than INR 20000'
        END AS price_band,
        COUNT(*) AS sku_count,
        ROUND(AVG(price), 2) AS avg_price_in_band,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products), 2) AS percentage
    FROM products
    GROUP BY price_band
    ORDER BY 
        CASE price_band
            WHEN 'Below INR 3000' THEN 1
            WHEN 'INR 3000-4999' THEN 2
            WHEN 'INR 5000-9999' THEN 3
            WHEN 'INR 10000-14999' THEN 4
            WHEN 'INR 15000-19999' THEN 5
            ELSE 6
        END
    """
    price_distribution = pd.read_sql_query(query, conn)
    
    # Create bar chart with percentage
    fig = px.bar(
        price_distribution, 
        x='price_band', 
        y='sku_count', 
        title='SKU Distribution by Price Band',
        labels={'sku_count': 'Number of SKUs', 'price_band': 'Price Band'},
        text='percentage'
    )
    
    # Update traces to display the percentage inside the bars
    fig.update_traces(
        texttemplate='%{text}%', 
        textposition='inside',
        textfont_size=10  # Adjust font size as needed
    )
    fig.update_layout(
        uniformtext_minsize=12, 
        uniformtext_mode='hide'
    )
    fig.show()

    return price_distribution

# Call the function
price_distribution = analyze_price_distribution(conn)

In [6]:
def analyze_brand_relative_ranking(conn):
    """Calculate and visualize brand relative ranking."""
    query = """
    SELECT 
        brand,
        COUNT(*) AS sku_count,
        AVG(rank) AS relative_rank
    FROM 
        products
    GROUP BY 
        brand
    HAVING 
        COUNT(*) > 1
    ORDER BY 
        relative_rank ASC
    """
    relative_rank = pd.read_sql_query(query, conn)
    
    # Select the top 10 brands by relative rank (ascending)
    relative_rank_top_10 = relative_rank.nsmallest(10, 'relative_rank')
    
    # Horizontal bar chart for ranking
    fig = go.Figure(
        data=go.Bar(
            x=relative_rank_top_10['relative_rank'],
            y=relative_rank_top_10['brand'],
            orientation='h',
            text=relative_rank_top_10['relative_rank'].round(2),
            textposition='auto'
        )
    )
    
    fig.update_layout(
        title='Top 10 Brands: Relative Ranking',
        xaxis_title='Relative Rank',
        yaxis_title='Brand',
        yaxis=dict(autorange="reversed")
    )
    fig.show()
    
    return relative_rank

relative_rank = analyze_brand_relative_ranking(conn)


Note: In this analysis, relative rank represents the average position of a brand's products in search results. <br>A lower relative rank (closer to 1) indicates better performance because it signifies that the brand's products appear earlier in search results, making them more visible and accessible to customers.

In [7]:
def analyze_brand_relative_rating(conn):
    """Calculate and visualize brand relative rating."""
    query = """
    SELECT 
        brand,
        AVG(rating) AS relative_rating
    FROM 
        products
    GROUP BY 
        brand
    ORDER BY 
        relative_rating DESC
    """
    relative_rating = pd.read_sql_query(query, conn)
    
    # Select the top 10 brands by relative rating
    relative_rating_top_10 = relative_rating.nlargest(10, 'relative_rating')
    
    # Horizontal bar chart for ratings
    fig = go.Figure(
        data=go.Bar(
            x=relative_rating_top_10['relative_rating'],
            y=relative_rating_top_10['brand'],
            orientation='h',
            text=relative_rating_top_10['relative_rating'].round(2),
            textposition='auto'
        )
    )
    
    fig.update_layout(
        title='Top 10 Brands: Relative Rating',
        xaxis_title='Relative Rating',
        yaxis_title='Brand',
        yaxis=dict(autorange="reversed")
    )
    fig.show()

    return relative_rating

relative_rating = analyze_brand_relative_rating(conn)

In [8]:
# Brand Performance Analysis
def analyze_brand_performance(conn):
    """Analyze brand performance across price and rating dimensions."""
    query = """
    SELECT 
        brand,
        COUNT(*) as product_count,
        ROUND(AVG(price), 2) as avg_price,
        ROUND(AVG(rating), 2) as avg_rating,
        ROUND(AVG(rank), 2) as avg_rank
    FROM products
    GROUP BY brand
    ORDER BY product_count  DESC, avg_rating DESC
    """
    performance = pd.read_sql_query(query, conn)
    
    # Select the top 10 brands by total products and average rating
    performance_top_10 = performance.nlargest(10, ['product_count', 'avg_rating'])
    
    # Bubble chart for performance visualization
    fig = px.scatter(performance_top_10, x='avg_price', y='avg_rating', 
                     size='product_count', color='brand',
                     hover_name='brand',
                     title='Brand Performance: Price vs Rating',
                     labels={'avg_price': 'Average Price', 
                             'avg_rating': 'Average Rating'},
                     size_max=60)
    fig.show()

    return performance

performance = analyze_brand_performance(conn)


In [9]:
# Merging the DataFrames
combined_df = pd.merge(brand_dist, relative_rating, on='brand')
combined_df = pd.merge(combined_df, relative_rank, on='brand')

# Creating a custom score for sorting competitors based on key metrics (without market share)
combined_df['score'] = combined_df['relative_rating'] * 0.4 + (1 / combined_df['relative_rank']) * 0.3 + (combined_df['product_count'] / combined_df['product_count'].sum()) * 0.3

# Sorting by performance score
top_competitors = combined_df.sort_values(by='score', ascending=False).head(5)

# Visualize the top competitors using a bar chart
import plotly.express as px

fig = px.bar(
    top_competitors,
    x='brand',
    y='score',
    title='Top 5 Competitors Based on Combined Metrics',
    labels={'score': 'Score'}
)
fig.show()

##### Explanation - The **'score'** used to rank the top competitors is calculated using key metrics:
- **Average rating** (40% weight): Higher average ratings indicate better product quality.
- **Relative rank** (30% weight): The inverse of the average product position in search results. A lower relative rank (closer to 1) signifies better visibility in search results.
- **Total products** (30% weight): Indicates brand presence, normalized by the total number of products to maintain comparability.

This approach focuses on product ratings, visibility, and brand presence.

## Key Insights and Recommendations

### Market Landscape
- **Total Brands**: 108 brands competing in the smart lock segment, showcasing significant market diversity.
- **Product Diversity**: 247 unique smart lock products, indicating a competitive landscape with varied options for consumers.

### Pricing Strategy
- **Price Range**: The majority of products are priced between INR 5,000 - 14,999, suggesting that this price range is the most attractive to consumers.
- **Average Market Price**: ₹9,866.56, highlighting a moderately premium market positioning that balances affordability and feature richness.

### Performance Metrics
- **Average Rating**: 3.5/5, which reflects moderate customer satisfaction across the market. This suggests room for improvement in product quality, design, and user experience.

### Top Competitors Analysis
- The **top-performing brands** have distinguished themselves through a combination of competitive pricing, innovative design, and high user ratings. Brands like **Godrej**, **Qubo**, and **Yale** have emerged as market leaders, leveraging strong customer satisfaction, robust feature sets, and effective marketing strategies to capture consumer interest.

### Recommendations for Circuit House
1. **Focus on mid-range pricing** (INR 5,000 - 14,999): This price range aligns with the majority of products in the market and positions Circuit House as a competitive option.
2. **Aim for product quality exceeding the current market average rating**: Enhancing product quality to score above the average 3.5/5 can increase customer satisfaction and brand reputation.
3. **Analyze top-performing brands**: Study the design, features, and customer feedback of leading competitors to identify key strengths and areas for differentiation.
4. **Innovate in product features**: Incorporate unique and high-demand features to attract consumers and stand out in the competitive market.

In [10]:
# Close database connection
if conn:
    conn.close()
    print("Database connection closed.")

Database connection closed.
